In [1]:
import os

os.chdir("../")

In [9]:
%pwd

'd:\\Project\\ETE_Projects\\image_classifer\\Mushroom_Edibility_Classifier'

In [10]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

[2025-06-07 23:35:10,668: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


In [16]:
from dataclasses import dataclass
from pathlib import Path
from typing import List


@dataclass
class EvaluationConfig:
    path_of_model: str
    test_data: str
    all_params: dict
    params_image_size: List[int]
    params_batch_size: int

In [8]:
from Mushroom_edibility_classifier.constants import *
from Mushroom_edibility_classifier.utils.common import read_yaml, create_directories,save_json

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config["artifacts_root"]])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=self.config["training"]["trained_model_path"],
            test_data=self.config["training"]["test_data"],  # uses "val" as test
            all_params=self.params,
            params_image_size=self.params["IMAGE_SIZE"],
            params_batch_size=self.params["BATCH_SIZE"]
        )
        return eval_config


In [4]:
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _test_generator(self):
        datagen_kwargs = dict(rescale=1. / 255)
        flow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode='categorical',
            shuffle=False
        )

        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

        self.test_generator = test_datagen.flow_from_directory(
            directory=self.config.test_data,
            **flow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._test_generator()
        self.score = self.model.evaluate(self.test_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


In [20]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluate()
    evaluation.save_score()

except Exception as e:
   raise e

[2025-06-07 23:50:16,663: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-07 23:50:16,683: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-07 23:50:16,684: INFO: common: created directory at: artifacts]


[2025-06-07 23:50:17,007: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 282 images belonging to 2 classes.


d:\Project\ETE_Projects\image_classifer\Mushroom_Edibility_Classifier\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


36/36 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9238 - loss: 0.4711
[2025-06-07 23:50:59,821: INFO: common: json file saved at: scores.json]
